<a href="https://colab.research.google.com/github/Shamiul-693/-Exploratory-Data-Analysis-and-Classification-Algorithms-on-Mushroom-Dataset/blob/main/Sherlock_Holmes_dataset_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import os
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import random